In [1]:
import datetime
import requests
import os
import json
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [2]:
work_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials\baidu_index' ##work_path must be adjusted before running the code.
os.chdir(work_path)

In [3]:
country_name=["加拿大",'智利','墨西哥','台湾','香港','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','夏威夷','奥地利','捷克']

In [4]:
class TimeSpan:
    def __init__(self,start,end):
        self.start=start.to_pydatetime()
        self.end=end.to_pydatetime()
    def get_span(self):
        return (self.end-self.start).days

In [5]:
def time_overlap(span1,span2):
    if not (span2.end<span1.start or span2.start>span1.end):
        overlap_start=max(span1.start,span2.start)
        overlap_end=min(span1.end,span2.end)
        return (overlap_end - overlap_start).days + 1
    else:
        return 0

In [ ]:
###the original baidu search index is collected on a weekly basis,here it is converted into monthly-based format.

for country in country_name: 
    temp=pd.read_excel(country+'.xlsx').iloc[:,1:]
    temp=temp.fillna(method='backfill')
    modify=pd.DataFrame()
    modify['date']=pd.date_range(start="2011-01", end="2023-05", freq="MS")
    for q in range(1,temp.shape[1]):
        name=temp.columns[q]
        i=0
        j=0
        lis=[]
        val=0
        span1=TimeSpan(temp.iloc[0,0],temp.iloc[1,0]-datetime.timedelta(days=1))
        span2=TimeSpan(modify.iloc[0,0],modify.iloc[1,0]-datetime.timedelta(days=1))
        while i<temp.shape[0]-1:
            span1=TimeSpan(temp.iloc[i,0],temp.iloc[i+1,0]-datetime.timedelta(days=1))
            overlap_days=time_overlap(span1,span2)
            month_length=span2.get_span()+1
            val+=temp.iloc[i,q]*overlap_days/month_length
            if span1.end>span2.end: 
                lis.append(round(val,2))
                val=0
                j+=1
                if j<148:
                    span2=TimeSpan(modify.iloc[j,0],modify.iloc[j+1,0]-datetime.timedelta(days=1))
                else:
                    span2=TimeSpan(modify.iloc[148,0],pd.to_datetime('2023-05-31'))
            else:
                i+=1
        val+=temp.iloc[-1,1]*3/31
        lis.append(val)
        modify[name]=lis
    modify.to_excel(country+'_月度.xlsx')

In [11]:
###construct the composite search index
###the keywords whose correlation efficient with tourist arrival is less than 0.6 is deleted
data=pd.read_excel(r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials\data.xlsx').iloc[:,1:]
data.index=pd.date_range(start="1989-01", end="2023-02", freq="MS")
for i,country in enumerate(country_name):
    df=pd.read_excel(country+'_月度.xlsx').iloc[:,1:]
    temp=df.copy()
    temp=temp.set_index('date')
    lis=list(data.iloc[264:,i])
    lis.extend([np.nan,np.nan,np.nan])
    temp['tourism_arrival']=lis
    temp['tourism_arrival']=temp['tourism_arrival'].shift(-1)
    temp=temp.dropna()
    ind=[]
    for i in range(temp.shape[1]-1):
        if pearsonr(temp.iloc[:,i],temp.iloc[:,-1])[0]>=0.6:
            ind.append(i)
        else:
            print(temp.columns[i])
    ind=[i+1 for i in ind]
    composite_index=df.iloc[:,ind].sum(axis=1)
    file=pd.DataFrame(columns=['date','composite_search_index','tourism_arrival'])
    file['date']=pd.date_range(start="2011-02", end="2023-06", freq="MS")
    file['tourism_arrival']=lis
    file['tourism_arrival']=file['tourism_arrival'].shift(-1)
    file['composite_search_index']=composite_index
    path=r'D:\university\研究生\tourism_forecasting_competition\submit\composite_search_index'
    # file.to_excel(path+'\\'+country+'.xlsx')


加拿大酒店
蒙特利尔
渥太华
圣地亚哥
香港旅游
日本酒店
日本小吃
东京
釜山
澳门签证
马尔代夫酒店
新加坡酒店
新西兰酒店
美国酒店
纽约
华盛顿
泰国旅游攻略
伊斯坦布尔
澳大利亚旅游
澳大利亚酒店


In [10]:
temp

,捷克旅游,捷克签证,tourism_arrival
date,,,
2011-12-01,102.10,79.10,9107.0
2012-01-01,112.19,89.48,8394.0
2012-02-01,136.66,115.48,10751.0
2012-03-01,160.39,121.13,17670.0
2012-04-01,132.77,119.00,24155.0
...,...,...,...
2022-07-01,23.87,90.06,10473.0
2022-08-01,33.26,91.26,7977.0
2022-09-01,32.40,83.30,7180.0
